In [1]:
import os
import gc
from configparser import ConfigParser
import datetime

import polars as pl

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as L
from pytorch_lightning.loggers.neptune import NeptuneLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from torchmetrics import F1Score, Accuracy

import sys
sys.path.append('..')

from models.code import CRNN, CNN, CBiRNN, CTransformer

In [2]:
config = ConfigParser()
config.read(['configs/main.ini', 'configs/main_secret.ini'], encoding='utf-8')

cfg = {
    'debug': True,
    'seed': 123,
    'device': 'gpu' if torch.cuda.is_available() else 'cpu',
    'n_devices': torch.cuda.device_count() if torch.cuda.is_available() else 1,
    'companies': ['CHMF', 'GAZP', 'GMKN', 'LKOH', 'MOEX', 'NLMK', 'NVTK', 'PLZL', 'ROSN', 'SBER', 'SNGS', 'T', 'TATN'],

    'batch_size': 32,
    'epochs': 100,
    'accamulate_grad_batches': 1,
    'lr': 3e-4,
    'mixed': True,
    'patience': 10,
    'save_top_k': 5,

    'weight_decay': 1e-2,
    'max_grad_norm': 1e6,
    'batch_scheduler': False,
    'scheduler': None,
    'scheduler_params': {'eta_min': 1e-5},

    'model_name': 'CNN',
    'num_workers': 4,
    'out_dir': 'D:/Jora/Аспирантура/Stock_forecast_with_investors_sentiment/models/weights/'
}

models = {
    'CNN': CNN,
    'CRNN': CRNN,
    'CBiRNN': CBiRNN,
    'CTransformer': CTransformer
}

base_model = models[cfg['model_name']]()

if cfg['debug']:
    kwargs = {'limit_train_batches': 2,
              'limit_val_batches': 2, 
              'num_sanity_val_steps': 0}
    cfg['batch_size'] = 2
    cfg['epochs'] = 2
    cfg['mixed'] = False if cfg['device'] == 'cpu' else True
else:
    kwargs = {}

os.makedirs(cfg['out_dir'] + cfg['model_name'], exist_ok=True)

In [3]:
class PulseSent(L.LightningModule):

    def __init__(self, model, lr, epochs, scheduler=None, scheduler_params=None, out_dir=cfg['out_dir']):
        super(PulseSent, self).__init__()

        self.model = model
        self.lr = lr
        self.epochs = epochs
        self.loss = nn.MSELoss()
        dev = 'cuda' if cfg['device'] == 'gpu' else 'cpu'
        self.metrics = {'f1': F1Score(task='binary').to(dev), 'acc': Accuracy(task='binary').to(dev)}
        self.scheduler = scheduler
        self.scheduler_params = scheduler_params
        self.out_dir = out_dir

    def forward(self, x):
        return self.model(x)

    def all_gather_reshape(self, data_lst):
        """Collects tensors from all GPUs and reshapes them to (NUM_GPUS*N, D, H, W)"""
        data_lst = list(self.all_gather(data_lst))
        for i in range(len(data_lst)):
            data_lst[i] = data_lst[i].reshape(-1, *data_lst[i].shape[2:])
        return data_lst

    def training_step(self, batch):
        x, y = batch
        out = self(x)
        loss = self.loss(out, y)
        if self.scheduler:
            lr = self.lr_schedulers().get_last_lr()[0]
            self.log(f'f{self.fold}-fit/lr', lr, on_step=False,
                     on_epoch=True, sync_dist=True)
        self.log(f"train_loss_{self.fold}", 100*loss.sqrt(), on_step=False,
                 on_epoch=True, prog_bar=True, sync_dist=True)
        return loss

    def validation_step(self, batch):
        x, y = batch
        out = self(x)
        val_loss = self.loss(out, y)
        if cfg['n_devices'] > 1:
            out, y = self.all_gather_reshape((out, y))
        for metric in self.metrics:
            self.metrics[metric].update(
                (out > 0).type(torch.int), (y > 0).type(torch.int))
            self.log(f"val_{metric}_{self.fold}", self.metrics[metric].compute(
            ), on_step=False, on_epoch=True, logger=True, prog_bar=True, sync_dist=True)
        self.log(f"val_loss_{self.fold}", 100*val_loss.sqrt(), on_step=False,
                 on_epoch=True, logger=True, prog_bar=True, sync_dist=True)

    def predict_step(self, x):
        return self(x)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(), lr=self.lr, weight_decay=cfg['weight_decay'])
        if self.scheduler:
            scheduler_dict = {
                "scheduler": self.scheduler(optimizer, **self.scheduler_params),
                'interval': 'step' if cfg['batch_scheduler'] else 'epoch',
                'frequency': 1
            }
            return {"optimizer": optimizer, "lr_scheduler": scheduler_dict}
        else:
            return optimizer

In [32]:
class SentDataset(Dataset):

    def __init__(self, df_train: pl.DataFrame, n_lags: int, df_val: pl.DataFrame=None, target_word='diff'):
        super().__init__()
        df_with_lags = self.create_lags(df_train, n_lags, df_val)

        target_cols = [col for col in df_with_lags.columns if target_word in col and 'lag' not in col]
        emb_cols = [col for col in df_with_lags.columns if 'column' in col and 'lag' in col]
        ts_cols = [col for col in df_with_lags.columns if target_word in col and 'lag' in col]

        embs = df_with_lags.select(emb_cols).to_torch(dtype=pl.Float32).view((len(df_with_lags), n_lags, -1))
        ts = df_with_lags.select(ts_cols).to_torch(dtype=pl.Float32).view((len(df_with_lags), n_lags, -1))

        self.X = torch.cat([embs, ts], dim=-1)
        self.y = df_with_lags.select(target_cols).to_torch(dtype=pl.Float32)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, ind):
        return self.X[ind], self.y[ind]
    
    @staticmethod
    def create_lags(df_train: pl.DataFrame, n_lags: int, df_val: pl.DataFrame=None):
        price_cols = [col for col in df_train.columns if 'diff' in col]
        emb_cols = [col for col in df_train.columns if 'column' in col]

        if df_val is not None:
            val_dates = df_val['date']
            df_train = pl.concat([df_train, df_val])
        
        lag_expressions = []

        for lag in range(1, n_lags + 1):
            for pc in price_cols:
                lag_expressions.append(
                    pl.col(pc).shift(lag).alias(f'{pc}_lag_{lag}')
                )
        
        for lag in range(1, n_lags+1):
            for ec in emb_cols:
                lag_expressions.append(
                    pl.col(ec).shift(lag).alias(f'{ec}_lag_{lag}')
                )

        df_with_lags = df_train.with_columns(lag_expressions)

        if df_val is not None:
            df_with_lags = df_with_lags.filter(pl.col('date').is_in(val_dates.implode())).sort('date')
        
        return df_with_lags.drop_nulls()

In [33]:
df = pl.read_parquet('../data/final/GAZP.parquet').drop(['weighted_1d', 'open', 'close', 'high', 'low'])

df_train = df.filter(pl.col('date') <= datetime.date(2024, 4, 30))
df_val = df.filter(pl.col('date') > datetime.date(2024, 4, 30),
                   pl.col('date') <= datetime.date(2024, 10, 31))
df_test = df.filter(pl.col('date') > datetime.date(2024, 10, 31))

In [53]:
ds_train = SentDataset(df_train, 10)
ds_val = SentDataset(df_train, 10, df_val)
ds_test = SentDataset(df_val, 10, df_test)

dl_train = DataLoader(ds_train, batch_size=32, shuffle=True, drop_last=True, pin_memory=True)
dl_val = DataLoader(ds_val, batch_size=32, shuffle=False, pin_memory=True)
dl_test = DataLoader(ds_test, batch_size=32, shuffle=False, pin_memory=True)

In [54]:
for batch in dl_train:
    break

batch[0].shape, batch[1].shape

(torch.Size([32, 10, 389]), torch.Size([32, 5]))